In [ ]:
import pandas as pd
import os
from datetime import datetime

# Użytkownik podaje datę (format: RRRR-MM-DD)
data_str = input("Podaj datę w formacie RRRR-MM-DD (np. 2025-07-01): ").strip()

# Tworzenie dynamicznych ścieżek do plików
airly_path = f"../data/processed/airly/airly_{data_str}.csv"
meteo_path = f"../data/processed/meteo/meteo_{data_str}.csv"
output_path = f"../data/combined/combined_{data_str}.csv"


In [6]:
def clean_airly(df: pd.DataFrame) -> pd.DataFrame:
    # Jeśli plik nie ma nagłówków lub brakuje kolumny 'name', nadaj je ręcznie
    if 'name' not in df.columns:
        df.columns = ['fromDateTime', 'tillDateTime', 'name', 'value']
    
    print('Kolumny po wczytaniu:', df.columns)
    print(df.head())
    
    # Usuwamy strefę czasową, zostawiamy tylko datę i godzinę
    df['fromDateTime'] = pd.to_datetime(df['fromDateTime']).dt.tz_localize(None)
    
    # Pivot: zamieniamy name -> kolumny
    df = df.pivot_table(index='fromDateTime', columns='name', values='value', aggfunc='mean')
    
    # Reset index, by datetime był zwykłą kolumną
    df = df.reset_index()
    
    # Zmiana nazw kolumn na małe litery itp.
    df = df.rename(columns={
        'fromDateTime': 'datetime',
        'PM1': 'pm1',
        'PM25': 'pm25',
        'PM10': 'pm10',
        'PRESSURE': 'pressure',
        'HUMIDITY': 'humidity',
        'TEMPERATURE': 'temperature',
    })
    
    df['source'] = 'airly'
    return df

In [7]:
def clean_meteo(df: pd.DataFrame) -> pd.DataFrame:
    """
    Czyści dane pogodowe z Open-Meteo:
    - Konwertuje time na datetime
    - Zmienia nazwy kolumn na lowercase
    - Dodaje sufiks _meteo
    """
    df['datetime'] = pd.to_datetime(df['time'])
    df = df.rename(columns={
        'temperature_2m': 'temperature',
        'relative_humidity_2m': 'humidity',
        'windspeed_10m': 'windspeed',
    })

    # Wybór kolumn i dodanie sufiksu
    df = df[['datetime', 'temperature', 'humidity', 'windspeed']]
    df = df.rename(columns={
        'temperature': 'temperature_meteo',
        'humidity': 'humidity_meteo',
        'windspeed': 'windspeed_meteo'
    })
    return df



In [8]:
try:
    # Wczytanie surowych danych
    df_airly_raw = pd.read_csv(airly_path)
    df_meteo_raw = pd.read_csv(meteo_path)
    
    # Czyszczenie
    df_airly = clean_airly(df_airly_raw)
    df_meteo = clean_meteo(df_meteo_raw)

    # Połączenie po datetime (tylko wspólne wiersze)
    df_combined = pd.merge(
    df_airly, df_meteo,
    on='datetime',
    how='inner'
    ).sort_values('datetime')

    # Tworzenie katalogu jeśli nie istnieje
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    # Zapis
    
    df_combined.to_csv(output_path, index=False)
    print(f"✅ Dane połączone i zapisane do: {output_path}")
except FileNotFoundError as e:
    print(f"❌ Błąd: Plik {e.filename} nie został znaleziony. Sprawdź ścieżki.")
except pd.errors.EmptyDataError:
    print("❌ Błąd: Plik jest pusty lub nie zawiera danych.")
except Exception as e:
    print(f"❌ Wystąpił błąd: {e}")


Kolumny po wczytaniu: Index(['fromDateTime', 'tillDateTime', 'name', 'value'], dtype='object')
               fromDateTime              tillDateTime      name    value
0  2025-07-02T18:36:59.276Z  2025-07-02T19:36:59.276Z       PM1     3.24
1  2025-07-02T18:36:59.276Z  2025-07-02T19:36:59.276Z      PM25     4.52
2  2025-07-02T18:36:59.276Z  2025-07-02T19:36:59.276Z      PM10     6.56
3  2025-07-02T18:36:59.276Z  2025-07-02T19:36:59.276Z  PRESSURE  1016.89
4  2025-07-02T18:36:59.276Z  2025-07-02T19:36:59.276Z  HUMIDITY    48.94
✅ Dane połączone i zapisane do: ../data/combined/combined_2025-07-02.csv
